In [ ]:
# Fastapi 연결용 웹소켓 새 로직
# STEP 1. 필수 라이브러리 설치
!pip install fastapi uvicorn websockets pyngrok nest-asyncio ultralytics

# STEP 2. ngrok 토큰 등록
from pyngrok import conf, ngrok
conf.get_default().auth_token = "xx"  # 토큰 입력

# STEP 3. YOLO 및 기타 모듈 임포트
import base64
import numpy as np
import cv2
import nest_asyncio
import uvicorn
import asyncio
import requests
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from ultralytics import YOLO
import threading #테스트용
import time

# STEP 4. YOLO 모델 로드
model = YOLO('yolov8n.pt')

FOCAL_LENGTH = 600
REAL_CAR_WIDTH = 1.3  # meters

FASTAPI_URL = "https://xx.ngrok-free.app/drive/event"  # 실제 FastAPI 주소

# STEP 5. FastAPI 서버 정의
app = FastAPI()


# 상태 변수 (전역)
was_safe = True  # 처음엔 안전한 상태

@app.websocket("/")
async def websocket_endpoint(websocket: WebSocket):
    global was_safe  # 상태 변수 수정하려면 global 필요
    await websocket.accept()
    print("WebSocket 연결됨")

    try:
        while True:
            img_bytes = await websocket.receive_bytes()

            # 이미지 디코딩
            nparr = np.frombuffer(img_bytes, np.uint8)
            frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            if frame is None:
                print("디코딩 실패")
                continue

            # YOLO 추론
            results = model(frame)
            unsafe_now = False

            for result in results:
                for box in result.boxes:
                    cls = int(box.cls[0])
                    if cls == 2 or cls == 7:  # 자동차 or 트럭
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        car_width = x2 - x1
                        if car_width > 0:
                            distance = (REAL_CAR_WIDTH * FOCAL_LENGTH) / car_width
                            if distance <= 4.0:
                                unsafe_now = True
                                break

            # 상태 변화 감지
            if unsafe_now and was_safe:
                print("Unsafe distance detected! (first entry)")
                was_safe = False
                try:
                    requests.post(FASTAPI_URL, json={"result": "unsafe distance"})
                except Exception as e:
                    print(f"FastAPI로 전송 실패: {e}")
            elif not unsafe_now:
                was_safe = True  # 다시 안전 상태로 복귀

    except WebSocketDisconnect:
        print("WebSocket 연결 종료됨")

# STEP 6. ngrok 및 uvicorn 실행
nest_asyncio.apply()
public_url = ngrok.connect(8000, bind_tls=True)
print(f"외부 WebSocket 주소: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
#Fastapi 연결용 웹소켓 <구 로직_실행안정성 확보>
# STEP 1. 필수 라이브러리 설치
!pip install fastapi uvicorn websockets pyngrok nest-asyncio ultralytics

# STEP 2. ngrok 토큰 등록
from pyngrok import conf, ngrok
conf.get_default().auth_token = "!!token!!"  # 토큰 입력

# STEP 3. YOLO 및 기타 모듈 임포트
import base64
import numpy as np
import cv2
import nest_asyncio
import uvicorn
import asyncio
import requests
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from ultralytics import YOLO
import threading #테스트용
import time

# STEP 4. YOLO 모델 로드
model = YOLO('yolov8n.pt')

FOCAL_LENGTH = 600
REAL_CAR_WIDTH = 1.3  # meters

FASTAPI_URL = "https://6xx.ngrok-free.app/drive/event"  # 실제 FastAPI 주소

# STEP 5. FastAPI 서버 정의
app = FastAPI()

@app.websocket("/")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    print("✅ WebSocket 연결됨")
    try:
        while True:
            img_bytes = await websocket.receive_bytes()

            # 이미지 디코딩
            nparr = np.frombuffer(img_bytes, np.uint8)
            frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            if frame is None:
                print(" 디코딩 실패")
                continue

            # YOLO 추론
            results = model(frame)
            unsafe_detected = False

            for result in results:
                for box in result.boxes:
                    cls = int(box.cls[0])
                    if cls == 2 or cls == 7:  # 자동차 or 트럭
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        car_width = x2 - x1
                        if car_width > 0:
                            distance = (REAL_CAR_WIDTH * FOCAL_LENGTH) / car_width
                            if distance <= 4.0:
                                print(" Unsafe distance detected!")
                                unsafe_detected = True
                                break


            # FastAPI로 결과 전송
            if unsafe_detected:
                try:
                    requests.post(FASTAPI_URL, json={"result": "unsafe distance"})
                except Exception as e:
                    print(f"FastAPI로 전송 실패: {e}")

    except WebSocketDisconnect:
        print("WebSocket 연결 종료됨")

async def event_sender():
    try:
        async with websockets.connect(EVENT_WS_URL) as websocket:
            print("이벤트 WebSocket 연결됨")
            while True:
                if unsafe_detected:
                    await websocket.send(json.dumps({"event": "unsafe distance"}))
                    print("이벤트 전송 완료: unsafe distance")
                await asyncio.sleep(0.1)
    except Exception as e:
        print(f"WebSocket 전송 실패: {e}")

# 쓰레드로 실행
threading.Thread(target=send_test_event_loop, daemon=True).start()


# STEP 6. ngrok 및 uvicorn 실행
nest_asyncio.apply()
public_url = ngrok.connect(8000, bind_tls=True)
print(f"외부 WebSocket 주소: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)
